# IR DE UN WAYPOINT AL SIGUIENTE Y PARAR

In [1]:
import glob
import os
import sys
import carla
import argparse
import random
import time
import math
import numpy as np
import cv2

#### CREAR MUNDO

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
client.load_world("Town05")
world = client.get_world()
map = world.get_map()

#### CAMBIO DE VISTA

In [8]:
loc2=carla.Transform(carla.Location(x=-174.557922, y=185.525208, z=10.138501), carla.Rotation(pitch=0.733207, yaw=-147.194565, roll=0.006472))
spectator = world.get_spectator()
spectator_pos = carla.Transform(loc2.location + carla.Location(x=10,y=5,z=4),carla.Rotation(yaw=loc2.rotation.yaw))
spectator.set_transform(spectator_pos)

#### CREAR VEHÍCULO

In [14]:
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')

vehicle = world.spawn_actor(bp, loc2)

#### VELOCIDAD CONSTANTE
Función para llevar una velocidad constante

In [6]:
def maintain_speed(s,preferred_speed,speed_threshold):
    #el return es la aceleración
    if s>=10:
        return 0
    elif s < 10 - 2:
        return 0.8
    else:
        return 0.4

#### ANGULO
Función que calcula el ángulo entre dos vectores

In [7]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    print(car_x)
    print(car_y)
    print(wp_x)
    print(wp_y)
    
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))
    return degrees

In [10]:
#Calculo del giro
def steer(predicted_angle,MAX_STEER_DEGREES):
    if predicted_angle<-180:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 180:
        predicted_angle = predicted_angle-360
    steer_input = predicted_angle
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    steer_input = steer_input/75
    return steer_input

#### LOCALIZACION INICIAL Y NEXT
Saca por pantalla la localización en la que estamos y el siguiente punto unos metros más delante y en el mismo carril

In [25]:
print("Initial location: ", vehicle.get_transform())
loc=vehicle.get_location()
current_w = map.get_waypoint(loc)
#Calculo del siguiente punto al que ir
waypoint_separation = 5
next_w0 = list(current_w.next(waypoint_separation))[0]
next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
print("next: ",next_w.transform)

Initial location:  Transform(Location(x=-207.346069, y=163.224777, z=9.402478), Rotation(pitch=0.547897, yaw=-133.580383, roll=-0.008636))
next:  Transform(Location(x=-209.589508, y=159.472366, z=9.445031), Rotation(pitch=360.530823, yaw=230.039185, roll=0.000000))


#### IMPLEMENTAR ÁNGULO Y VELOCIDAD AL COCHE 

In [26]:
preferred_speed = 10
speed_threshold = 2
MAX_STEER_DEGREES = 5
velocity = vehicle.get_velocity()
speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
estimated_throttle = maintain_speed(speed,preferred_speed,speed_threshold)
predicted_angle = get_angle(vehicle,next_w)
steer_input=steer(predicted_angle,MAX_STEER_DEGREES)
#Aplicamos los cálculos al coche
vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steer_input))
while True:
    if vehicle.get_transform().location.x<= next_w.transform.location.x and vehicle.get_transform().location.y<= next_w.transform.location.y:
        vehicle.apply_control(carla.VehicleControl(brake=1.0,throttle= 0, steer=0))
        print("Hemos llegado a next")
        break

-207.3460693359375
163.2247772216797
-209.58950805664062
159.4723663330078
Hemos llegado a next


### DESTRUIR

In [27]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()